<a href="https://colab.research.google.com/github/SiriosMEN/Data-science/blob/main/bootstrap/bootstrap_oil_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

В исследовании загрузим данные из датасетов, проведем анализ и предподготовку данных.
Далее создадим и обучим модели для предсказания запасов нефти по известным данным разведки.
подготовим функцию для расчета прибыльности разработки месторождений в регионе. Методикой bootstrap создадим выборки для оценки прибыльности по предсказанным запасам. Выберем предпочтительный регион для разработки.

## Загрузка и подготовка данных

Загрузим данные из датасетов:

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
from scipy import stats as st

In [ ]:
data_1 = pd.read_csv('/datasets/geo_data_0.csv')
data_2 = pd.read_csv('/datasets/geo_data_1.csv')
data_3 = pd.read_csv('/datasets/geo_data_2.csv')

In [ ]:
data_1.info();
data_1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [ ]:
data_2.info();
data_2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [ ]:
data_3.info()
data_3.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Проверим распределение данных в таблицах, наличие выбросов:

In [ ]:
data_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [ ]:
data_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [ ]:
data_3.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Проверим корреляцию данных:

In [ ]:
data_1.corr()

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


In [ ]:
data_2.corr()

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


In [ ]:
data_3.corr()

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


В разных датасетах данные по разному коррелируют между собой. На датасете1 и датасете3 некоторые пары параметров коррелируют довольно сильно.

Колонка с id не должна влиять на предсказания. Удалим колонки из датасетов:

In [ ]:
data_1 = data_1.drop(['id'],axis='columns')
data_2 = data_2.drop(['id'],axis='columns')
data_3 = data_3.drop(['id'],axis='columns')

## Обучение и проверка модели

Создадим и обучим модели на базе алгоритма "линейная регрессия" для каждого региона.

In [ ]:
def train(data,region):
    features = data.drop('product',axis='columns')
    target = data['product']
    features_train,features_valid,target_train,target_valid = train_test_split(features,target,test_size=0.25,random_state=12345)
    model = LinearRegression(normalize=True)
    model.fit(features_train,target_train)
    predictions = model.predict(features_valid)
    mean_fact = data['product'].mean()
    mean = predictions.mean()
    RMSE = (mean_squared_error(target_valid,predictions))**0.5
    print("Средний запас фактический в регионе",region,":",mean_fact)
    print("Средний запас предсказанного сырья в регионе",region,":",mean)
    print("RMSE модели в регионе",region,":",RMSE)
    return mean_fact,predictions,target_valid, model

In [ ]:
mean_fact1, pred1,target_1, model_1 = train(data_1,1)
mean_fact2, pred2,target_2, model_2 = train(data_2,2)
mean_fact3, pred3,target_3, model_3 = train(data_3,3)

Средний запас фактический в регионе 1 : 92.50000000000001
Средний запас предсказанного сырья в регионе 1 : 92.59256778438035
RMSE модели в регионе 1 : 37.5794217150813
Средний запас фактический в регионе 2 : 68.82500000000002
Средний запас предсказанного сырья в регионе 2 : 68.728546895446
RMSE модели в регионе 2 : 0.8930992867756157
Средний запас фактический в регионе 3 : 95.00000000000004
Средний запас предсказанного сырья в регионе 3 : 94.96504596800489
RMSE модели в регионе 3 : 40.02970873393434


### Вывод

Модели по региону 1 и 3 имеют не очень хорошие показатели метрики RMSE. По региону 2 метрика неплохая. При этом по всем трем регионам среднее значение предсказанных запасов довольно близко к среднему известных данных датасетов.

## Подготовка к расчёту прибыли

Подготовим функцию для расчета прибыльности разработки региона.

In [ ]:
BUDGET = 10000000000
RESEARCHED_POINTS = 500
INCOME_PER_ONE_PRODUCT = 450000
SELECTED_POONTS_NUMBER = 200

Рассчитаем минимальный средний запас для безубыточности скважины

In [ ]:
min_mean_prod = BUDGET / (SELECTED_POONTS_NUMBER * INCOME_PER_ONE_PRODUCT)
print("Минимальный средний запас для безубыточности:",min_mean_prod)

Минимальный средний запас для безубыточности: 111.11111111111111


Минимальный средний запас намного превышает средние запасы по всем регионам.

In [ ]:
def profit_calc(data,model):
    features_p = data.drop('product',axis='columns')
    predictions_p = pd.Series(model.predict(features_p),index=features_p.index)
    selected_points =predictions_p.sort_values(ascending=False).head(SELECTED_POONTS_NUMBER)
    true_product = data.loc[(selected_points.index),'product']
    total_profit = sum(true_product * INCOME_PER_ONE_PRODUCT) - BUDGET
    return total_profit

### Вывод

Сохранены глобальные константы для расета прибыли. Написана функция для расчета прибыльности.

## Расчёт прибыли и рисков 

Рассчитаем прибыльность разработки скважин в каждом регионе, а также оценим риск убытков. Для оценки применим методику bootsstrap.

In [ ]:
def bootstrap(data,model,region):
    state = RandomState(12345)
    profit = []
    for i in range(1000):
        subsample = data.sample(RESEARCHED_POINTS,random_state = state,replace=True)
        profit.append(profit_calc(subsample,model))
    profit = pd.Series(profit)
    mean_profit = profit.mean().astype('int')
    confidence_interval_start = profit.quantile(0.025)
    confidence_interval_fin = profit.quantile(0.975)
    confidence_interval = [confidence_interval_start,confidence_interval_fin]
    confidence_interval_wide = confidence_interval_fin - confidence_interval_start
    risk_ases = 100 * (profit<0).sum() / len(profit)
    print("Средняя предполагаемая прибыль в регионе",region,":",mean_profit)
    print("Доверительный интервал в регионе",region,":",confidence_interval)
    print("Риск убытка в регионе",region,":",risk_ases,"%")
    return mean_profit,confidence_interval,confidence_interval_wide,risk_ases,profit

In [ ]:
mean_profit1,confidence_interval1,confidence_interval_wide1,risk_ases1,profit1 = bootstrap(data_1,model_1,1)
mean_profit2,confidence_interval2,confidence_interval_wide2,risk_ases2,profit2 = bootstrap(data_2,model_2,2)
mean_profit3,confidence_interval3,confidence_interval_wide3,risk_ases3,profit3 = bootstrap(data_3,model_3,3)
final_table = pd.DataFrame({'Регион':['1','2','3'],'Средняя прибыль':[mean_profit1,mean_profit2,mean_profit3],'Доверительный интервал':[confidence_interval1,confidence_interval2,confidence_interval3],'Ширина доверительного интервала':[confidence_interval_wide1,confidence_interval_wide2,confidence_interval_wide3],'Оценка риска убытков':[risk_ases1,risk_ases2,risk_ases3]})
display(final_table)

Средняя предполагаемая прибыль в регионе 1 : 472872289
Доверительный интервал в регионе 1 : [-62866509.83260307, 1051586773.0163103]
Риск убытка в регионе 1 : 4.3 %
Средняя предполагаемая прибыль в регионе 2 : 498762520
Доверительный интервал в регионе 2 : [67860923.08597451, 896096796.3533506]
Риск убытка в регионе 2 : 1.3 %
Средняя предполагаемая прибыль в регионе 3 : 424457434
Доверительный интервал в регионе 3 : [-158184644.98919243, 971736212.7329218]
Риск убытка в регионе 3 : 7.9 %


,Регион,Средняя прибыль,Доверительный интервал,Ширина доверительного интервала,Оценка риска убытков
0,1,472872289,"[-62866509.83260307, 1051586773.0163103]",1.114453e+09,4.3
1,2,498762520,"[67860923.08597451, 896096796.3533506]",8.282359e+08,1.3
2,3,424457434,"[-158184644.98919243, 971736212.7329218]",1.129921e+09,7.9


### Вывод

Риски убытков в первом и третьем регионах превышают 2,5%. Наибольшая средняя прибыль в регионе 2. По всем показателям второй регион наиболее предпочтителен для разработки.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован